In [ ]:
# In a Jupyter Notebook cell:

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# --- Reuse the PredatorPreyChemostat class from your .py file ---
# You can copy and paste it here, or if you save it as a separate module 
# (e.g., predator_prey_model.py) and import it. For now, let's assume
# you copy-paste it into a cell above this one.

# Assuming PredatorPreyChemostat class is already defined in a previous cell...

# Example: Copy-paste the class definition here if running in a fresh notebook
class PredatorPreyChemostat:
    """
    A class to encapsulate the predator-prey model in a chemostat,
    based on the paper "Crossing the Hopf Bifurcation in a Live Predator-Prey System."
    """
    def __init__(self, bc=3.3, Kc=4.3, bB=2.25, KB=15, m=0.055, lam=0.4, eps=0.25, delta=0.6, Ni=80.0):
        """
        Initializes the model with its parameters.
        Default values are taken from note 18 in the paper.
        """
        self.params = {
            'bc': bc, 'Kc': Kc, 'bB': bB, 'KB': KB,
            'm': m, 'lam': lam, 'eps': eps, 'delta': delta, 'Ni': Ni
        }

    def _model_equations(self, t, y):
        """
        Defines the system of four differential equations.
        """
        N, C, R, B = y
        p = self.params

        Fc = p['bc'] * N / (p['Kc'] + N)
        Fb = p['bB'] * C / (p['KB'] + C)

        dN_dt = p['delta'] * (p['Ni'] - N) - Fc * C
        dC_dt = Fc * C - (Fb * B / p['eps']) - p['delta'] * C
        dR_dt = Fb * R - (p['delta'] + p['m'] + p['lam']) * R
        dB_dt = Fb * R - (p['delta'] + p['m']) * B

        return [dN_dt, dC_dt, dR_dt, dB_dt]

    def run_simulation(self, y0, t_span, t_eval):
        """
        Runs the simulation using SciPy's solve_ivp.
        """
        sol = solve_ivp(
            self._model_equations, t_span, y0,
            t_eval=t_eval, method='RK45'
        )
        return {
            'time': sol.t, 'N': sol.y[0], 'Chlorella': sol.y[1],
            'Reproducing_Brachionus': sol.y[2], 'Total_Brachionus': sol.y[3]
        }
    
    def set_parameters(self, **kwargs):
        """Convenience function to update any model parameter."""
        for key, value in kwargs.items():
            if key in self.params:
                self.params[key] = value
            else:
                print(f"Warning: Parameter '{key}' not found.")

# --- Interactive Dashboard Setup ---

# Define the widgets for parameters
delta_slider = widgets.FloatSlider(
    value=0.6, min=0.05, max=1.75, step=0.01, description='Dilution Rate (δ):',
    continuous_update=False, readout_format='.2f'
)
Ni_slider = widgets.FloatSlider(
    value=80.0, min=20.0, max=500.0, step=10.0, description='Nitrogen Inflow (Ni):',
    continuous_update=False, readout_format='.1f'
)

# Other fixed parameters and initial conditions
fixed_params = {
    'bc': 3.3, 'Kc': 4.3, 'bB': 2.25, 'KB': 15,
    'm': 0.055, 'lam': 0.4, 'eps': 0.25
}
initial_conditions = [60.0, 10.0, 5.0, 5.0] # N, C, R, B
simulation_time = (0, 200) # Simulate for 200 days
t_eval_sim = np.linspace(simulation_time[0], simulation_time[1], 1000)

# Output widget to hold plots
output_plot = widgets.Output()

# Function to update the plot based on slider values
def update_plot(delta, Ni):
    with output_plot:
        clear_output(wait=True) # Clear previous plot
        
        # Create a system instance with current parameters
        system = PredatorPreyChemostat(**fixed_params, delta=delta, Ni=Ni)
        
        # Run the simulation
        results = system.run_simulation(initial_conditions, simulation_time, t_eval_sim)
        
        # Create the plot
        fig, ax1 = plt.subplots(figsize=(10, 5))

        # Time Series Plot
        color_prey = 'g'
        ax1.set_xlabel('Time (days)')
        ax1.set_ylabel('Prey Conc. (Chlorella)', color=color_prey)
        ax1.plot(results['time'], results['Chlorella'], color=color_prey, label='Chlorella')
        ax1.tick_params(axis='y', labelcolor=color_prey)
        ax1.grid(True)

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color_predator = 'k'
        ax2.set_ylabel('Predator Conc. (Brachionus)', color=color_predator)  # we already handled the x-label with ax1
        ax2.plot(results['time'], results['Total_Brachionus'], color=color_predator, label='Brachionus')
        ax2.tick_params(axis='y', labelcolor=color_predator)

        fig.suptitle(f'Predator-Prey Dynamics (δ = {delta:.2f}, Ni = {Ni:.1f})')
        fig.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
        plt.show()

# Link the sliders to the update function
interactive_plot = widgets.interactive_output(
    update_plot, 
    {'delta': delta_slider, 'Ni': Ni_slider}
)

# Display the widgets and the plot output
display(widgets.VBox([widgets.VBox([delta_slider, Ni_slider]), output_plot]))